In [ ]:
# !pip install category_encoders
# !pip install pdpbox
# !pip install eli5
# !pip install pandas-profiling==2.*
# !pip install shap
# !pip install xgboost==1.4.2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# 1.Load Data

## 1-1. Data description
멕시코 의대생 776명의 정신 건강 데이터

2014년 멕시코 사립 대학 4520명의 의대생 중 1200명을 무작위 샘플링. 의대생 776명 참여 동의(64.6% 응답).

- age                                     나이
- gender                                  성별
- height                                  키(m)
- weight                                  무게(kg)
- thoughts_of_dropping_out                자퇴를 심각하게 생각하였는지 여부
- previous_depression_diagnosis           이전에 우울증 진단을 받은적 있는지 여부
- previous_depression_treatment           이전에 우울증 치료를 받은적 있는지 여부
- previous_anxiety_diagnosis              이전에 불안 진단을 받은적 있는지 여부
- previous_anxiety_treatment              이전에 불안 치료를 받은적 있는지 여부
- bed_time                                자가 보고한 평소 취침 시간
- wake_up_time                            자가 보고한 평소 기상 시간
- reported_sleep_hours                    자가 보고한 체감 수면 시간
- times_week_nap                          일주일에 낮잠을 자는 횟수
- nap_duration                            자가 보고한 낮잠 시간
- weekly_study_hours                      자가 보고한 주간 학습 시간
- grades                                  자가 보고한 학점 평균(1.0-10.0)


- PHQ-9 우울증 척도
 - 0-4= no notable depressive symptoms
 - 5-9= mild
 - 10-14= moderate
 - 15-19= moderately severe
 - 20+= severe

- GAD-7 불안증 척도
 - 0-4= no notable anxiety symptoms
 - 5-9= mild
 - 10-14= moderate
 - 15+= severe

- EPW Epworth 졸음증 척도
 - 0-7:It is unlikely that you are abnormally sleepy.
 - 8-9:You have an average amount of daytime sleepiness.
 - 10-15:You may be excessively sleepy depending on the situation. You may want to consider seeking medical attention.
 - 16-24:You are excessively sleepy and should consider seeking medical attention

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# pd.set_option('display.max_rows', 776)
# pd.set_option('display.max_columns', 43)
df = pd.read_csv('/content/drive/MyDrive/AIB_11/mexican_medical_students_mental_health_data.csv')
df

In [ ]:
def analysis_on_features(df):
    eda_results = pd.DataFrame()
    eda_results["null_count"] = df.isnull().sum()
    eda_results["num_unique_values"] = df.nunique()
    duplicated_rows = df.duplicated().sum()
    return eda_results, duplicated_rows

In [ ]:
df_results, df_duplicated_rows = analysis_on_features(df)
print(f"df_duplicated? = {df_duplicated_rows}\n", df_results, "\n\n")

In [ ]:
df.dtypes

# 2.Data Preprocessing

## 2-1.PHQ-9 우울평가지표 타겟화

### 2-1-1.missing value

In [ ]:
phq = df.iloc[:,7:16]
gad = df.iloc[:,19:26]
epw = df.iloc[:,28:36]

In [ ]:
phq[phq.isnull().any(axis=1)]

In [ ]:
gad[gad.isnull().any(axis=1)]

In [ ]:
epw[epw.isnull().any(axis=1)]

In [ ]:
def find_nan_row(df):
    index_list = []
    for x in df.index:
        if df.loc[[x]].isna().sum().sum() > 1: # 한 행에 NaN값이 1개 초과 있는 index 저장
            index_list.append(x)
    return index_list

In [ ]:
# PHQ-9, GAD-7, EPW 각 평가지표 중 NaN값이 1개 초과인 행 삭제
index = list(set(find_nan_row(phq) + find_nan_row(gad) + find_nan_row(epw))) #set으로 중복 제거 후, 다시 list화

In [ ]:
df.drop(index=index, axis=0, inplace=True)

In [ ]:
phq = df.iloc[:,7:16]
gad = df.iloc[:,19:26]
epw = df.iloc[:,28:36]

In [ ]:
#fillna 함수와 역행렬을 이용하여 NaN값을 각 행의 중앙값으로 대체
phq = phq.T.fillna(phq.median(axis=1)).T
gad = gad.T.fillna(gad.median(axis=1)).T
epw = epw.T.fillna(epw.median(axis=1)).T

In [ ]:
print(f"{phq.isnull().sum()}\n{gad.isnull().sum()}\n{epw.isnull().sum()}")

### 2-1-2.mental score

In [ ]:
phq_score = phq.sum(axis=1)
df['phq_score'] = phq_score

gad_score = gad.sum(axis=1)
df['gad_score'] = gad_score

epw_score = epw.sum(axis=1)
df['epw_score'] = epw_score

In [ ]:
phq_col = phq.columns
gad_col = gad.columns
epw_col = epw.columns
df.drop(phq_col, inplace=True, axis=1)
df.drop(gad_col, inplace=True, axis=1)
df.drop(epw_col, inplace=True, axis=1)

df.drop(['id'], inplace=True, axis=1)

In [ ]:
# 평가지표 점수 초과(이상치)
print('PHQ-9 outlier:',(df['phq_score'] > 27).sum())
print('GAD-7 outlier:',(df['gad_score'] > 21).sum())
print('EPW outlier:',(df['epw_score'] > 24).sum())

In [ ]:
outlier_index = df[df['epw_score'] > 24].index
df.drop(index=outlier_index, axis=0, inplace=True)

In [ ]:
#PHQ-9 지표에 의해 15점 이상은 우울감이 존재.
df['phq_score'] = (df['phq_score'] >= 15).astype(int)

In [ ]:
df['phq_score'].value_counts() #데이터 불균형 확인

In [ ]:
plt.pie(
    (df['phq_score'].value_counts()[0], df['phq_score'].value_counts()[1]),
    labels=('Depression(X)', 'Depression(O)'),
    autopct="%.2f%%",
    shadow=True,
    startangle=90,
)
plt.title("Depression Ratio", size=15)
plt.show()

In [ ]:
df.head()

### 2-1-3. 불안증이 있는 사람중 우울증은 얼마나 있을까?

In [ ]:
check_gad = df[['gad_score']]
check_gad['check_phq'] = df['phq_score']
check_gad['gad_score'] = (check_gad['gad_score'] >= 15).astype(int)

In [ ]:
check_gad[check_gad['gad_score']==1]

In [ ]:
tmp = check_gad[check_gad['gad_score']==1]
tmp['check_phq'].value_counts()

In [ ]:
plt.pie(
    (tmp['check_phq'].value_counts()[0], tmp['check_phq'].value_counts()[1]),
    labels=('Depression(X)', 'Depression(O)'),
    autopct="%.2f%%",
    shadow=True,
    startangle=90,
)
plt.title("Depression Anxiety Ratio", size=15)
plt.show()

## 2-2.시간 관련 피쳐

### 2-2-1.오전 및 오후를 잘못 기입한 경우

In [ ]:
df['bed_time'].unique()

In [ ]:
df[df['bed_time'].isin({'12:00','09:30','05:00'})]

In [ ]:
#오후 시각과 오전 시각을 잘못 보고한 것으로 확인해서 수정.
df['bed_time'] = df['bed_time'].replace('12:00', '00:00')
df['bed_time'] = df['bed_time'].replace('09:30', '21:30')

### 2-2-2.주간 공부 시간

In [ ]:
df.groupby('weekly_study_hours')['grades'].agg('count')

In [ ]:
#주단위로 보고한 사람과 일단위로 보고한 사람이 섞여있는 것으로 판단되서 삭제
df.drop(['weekly_study_hours'], inplace=True, axis=1)

### 2-2-3.nap 관련 피쳐

In [ ]:
nap = df[['times_week_nap', 'nap_duration']]

In [ ]:
conditon_1 = nap[(nap['times_week_nap'].isnull()) & (nap['nap_duration'].isnull())]  #'times_week_nap', 'nap_duration' 두 컬럼 모두 NaN인 행 찾기
condition_1_index = conditon_1.index
conditon_1

In [ ]:
conditon_2 = nap[(nap['times_week_nap'] == 0) & (nap['nap_duration'].isnull())]  #'times_week_nap'는 0이고, 'nap_duration'는 NaN인 행 찾기
condition_2_index = conditon_2.index
conditon_2

In [ ]:
nap[(nap['times_week_nap'].isnull()) & (nap['nap_duration'] == 0)]

In [ ]:
index = conditon_1.index.tolist() + conditon_2.index.tolist()

In [ ]:
nap.loc[index,'times_week_nap'] = 0
nap.loc[index,'nap_duration'] = '00:00'

In [ ]:
nap[(nap['times_week_nap'].isnull()) | (nap['nap_duration'].isnull())]

In [ ]:
imputer = SimpleImputer(strategy = 'most_frequent') # 최빈값으로 대체
imputer.fit(nap)
nap[['times_week_nap','nap_duration']] = imputer.transform(nap[['times_week_nap','nap_duration']])

In [ ]:
nap.isnull().sum()

In [ ]:
df['times_week_nap'] = nap['times_week_nap']
df['nap_duration'] = nap['nap_duration']

### 2-2-4.실제 수면시간과 보고된 체감 수면시간

In [ ]:
sleep_time = df[['bed_time','wake_up_time','reported_sleep_hours']]
sleep_time_nan = sleep_time[(sleep_time['bed_time'].isnull()) | (sleep_time['wake_up_time'].isnull()) | (sleep_time['reported_sleep_hours'].isnull())]
sleep_time_nan

In [ ]:
condition = sleep_time_nan[sleep_time_nan.T.isnull().sum() > 1]
condition

In [ ]:
index = condition.index

In [ ]:
df.drop(index=index, axis=0, inplace=True)

In [ ]:
sleep_time = df[['bed_time','wake_up_time','reported_sleep_hours']]
sleep_time_nan = sleep_time[(sleep_time['bed_time'].isnull()) | (sleep_time['wake_up_time'].isnull()) | (sleep_time['reported_sleep_hours'].isnull())]
sleep_time_nan

In [ ]:
index = sleep_time_nan[sleep_time_nan.T.isnull().sum() == 1].index

In [ ]:
sleep_time = sleep_time.fillna('00:00') # 연산을 위해 임시로 데이터를 넣어놓고, 연산 후 해당 인덱스만 ratio를 median으로 채울예정

In [ ]:
sleep_time.isnull().sum()

In [ ]:
df['bed_time'] = sleep_time['bed_time']
df['wake_up_time'] = sleep_time['wake_up_time']
df['reported_sleep_hours'] = sleep_time['reported_sleep_hours']

### 2-2-5.time to minute

In [ ]:
# 데이터에 NaN값 있을때 처리방법
# def time2min(df):
#     hour = df.apply(lambda x: x if pd.isnull(x) else x.split(':')[0]).astype(int)
#     minute = df.apply(lambda x: x if pd.isnull(x) else x.split(':')[1]).astype(int)
#     h2m = hour * 60
#     time = h2m + minute
#     return time

In [ ]:
def time2min(df):
    hour = df.apply(lambda x: x.split(':')[0]).astype(int)
    minute = df.apply(lambda x: x.split(':')[1]).astype(int)
    h2m = hour * 60
    time = h2m + minute
    return time

In [ ]:
df['nap_duration'] = time2min(df['nap_duration'])
df['reported_sleep_hours'] = time2min(df['reported_sleep_hours'])

In [ ]:
import datetime
bed_hour = pd.to_datetime(df['bed_time']).dt.hour.apply(lambda x: -x if x <= 12 else 24-x)
wake_up_hour = pd.to_datetime(df['wake_up_time']).dt.hour

bed_min = pd.to_datetime(df['bed_time']).dt.minute
wake_up_min = pd.to_datetime(df['wake_up_time']).dt.minute

sleep_min = (bed_hour + wake_up_hour) * 60 + (wake_up_min - bed_min)
df['bed2wake_up_min'] = sleep_min

### 2-2-6.실제 수면시간과 보고된 체감 수면시간 비율

In [ ]:
df['sleep_ratio'] = (df['reported_sleep_hours'] / df['bed2wake_up_min']).round(2) #잠자리에 든 시간 대비 보고된 실 수면시간 비율

In [ ]:
df['sleep_ratio'].loc[index] = df['sleep_ratio'].median() # (NaN값이 있어 임시로 처리했던 데이터) sleep_ratio컬럼의 저장되었던 index들의 값을 median 값으로 수정

#### 2-2-6-1.이상치 조정

In [ ]:
tmp = df[['bed_time','wake_up_time','reported_sleep_hours','bed2wake_up_min','sleep_ratio','phq_score']]
tmp[tmp['sleep_ratio'] > 1]

In [ ]:
index = tmp[tmp['sleep_ratio'] > 1].index
df['sleep_ratio'].loc[index] = 1 # 수면을 취한 시간 대비 보고된 수면 시간 비율이 1 이상인 경우 비율 1로 통일.

In [ ]:
# 수면시간은 개인차가 있기때문에 sleep_ratio와 epw_score로만 수면의 질을 판단하기로 결정.
df.drop(['bed_time','wake_up_time','reported_sleep_hours','bed2wake_up_min'], inplace=True, axis=1)

## 2-3.gender

In [ ]:
df['gender'] = (df['gender'] == 'm').astype(int)

In [ ]:
df.head()

## 2-4.bmi

멕시코 bmi 기준 : https://www.bariatricmexico.com/bmi-calculator.html  
Underweight	<18.5  
Normal weight	18.5 - 24.9  
Overweight	25 - 29.9  
Obesity	BMI of 30 or greater  

In [ ]:
df['bmi'] = (df['weight'] / (df['height'] * df['height'])).astype(float).round(1)

In [ ]:
sns.boxplot(y = df['bmi'], orient = 'v')
plt.show()

In [ ]:
# bins = [0, 18.5, 25, 30, 60]
# bins_label = ['underweight','normal weight','overweight','obersity','outlier']

# bmi_cut = pd.cut(df['bmi'], bins, right=False, labels=bins_label[:-1])
# df_bmi = pd.get_dummies(bmi_cut).astype(int)
# df = pd.concat([df,df_bmi],axis=1)

In [ ]:
df.drop(['height','weight'], inplace=True, axis=1)

## 2-5.이상치 확인

In [ ]:
# filtered_data = df.dropna()
# columns = filtered_data.columns

# sns.set(rc = {'figure.figsize':(40,20)})
# fig, axes = plt.subplots(1,len(columns))
# for i in range(len(columns)):
#     sns.boxplot(y = filtered_data[columns[i]], data=filtered_data,  orient='v' , ax=axes[i])

## 2-6.data describe

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
def pieplot(df,custom_label):
    data = df.value_counts().values
    if custom_label == 0:
        labels = df.value_counts().index
    else:
        labels = custom_label

    colors = sns.color_palette('pastel')[0:5]

    fig = plt.figure(figsize=(8,8)) ## 캔버스 생성
    fig.set_facecolor('white') ## 캔버스 배경색을 하얀색으로 설정
    ax = fig.add_subplot() ## 프레임 생성

    pie = ax.pie(data, ## 파이차트 출력
        startangle=90, ## 시작점을 90도(degree)로 지정
        counterclock=False, ## 시계방향으로 그려짐
        colors = colors, ## 색상 지정
        rotatelabels =False
        )
    
    total = np.sum(data) ## 빈도수 합
    
    threshold = 5 ## 상한선 비율
    sum_pct = 0 ## 퍼센티지
    
    bbox_props = dict(boxstyle='square',fc='w',ec='w',alpha=0) ## annotation 박스 스타일
    
    config = dict(arrowprops=dict(arrowstyle='-'),bbox=bbox_props,va='center')
    
    for i,l in enumerate(labels):
        ang1, ang2 = ax.patches[i].theta1, ax.patches[i].theta2 ## 파이의 시작 각도와 끝 각도
        center, r = ax.patches[i].center, ax.patches[i].r ## 원의 중심 좌표와 반지름길이
        
        if i < len(labels) - 1:
            sum_pct += float(f'{data[i]/total*100:.2f}')
            text = f'{data[i]/total*100:.2f}%'
        else: ## 마지막 파이 조각은 퍼센티지의 합이 100이 되도록 비율을 조절
            text = f'{100-sum_pct:.2f}%'
        
        ## 비율 상한선보다 작은 것들은 Annotation으로 만든다.
        if data[i]/total*100 < threshold:
            ang = (ang1+ang2)/2 ## 중심각
            x = np.cos(np.deg2rad(ang)) ## Annotation의 끝점에 해당하는 x좌표
            y = np.sin(np.deg2rad(ang)) ## Annotation의 끝점에 해당하는 y좌표
            
            ## x좌표가 양수이면 즉 y축을 중심으로 오른쪽에 있으면 왼쪽 정렬
            ## x좌표가 음수이면 즉 y축을 중심으로 왼쪽에 있으면 오른쪽 정렬
            horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
            connectionstyle = "angle,angleA=0,angleB={}".format(ang) ## 시작점과 끝점 연결 스타일
            config["arrowprops"].update({"connectionstyle": connectionstyle}) ## 
            ax.annotate(text, xy=(x, y), xytext=(1.5*x, 1.2*y),
                        horizontalalignment=horizontalalignment, **config)
        else:
            x = (r/2)*np.cos(np.pi/180*((ang1+ang2)/2)) + center[0] ## 텍스트 x좌표
            y = (r/2)*np.sin(np.pi/180*((ang1+ang2)/2)) + center[1] ## 텍스트 y좌표
            ax.text(x,y,text,ha='center',va='center',fontsize=12)
        
    plt.legend(pie[0],labels,loc='upper right') ## 범례
    plt.show()

In [ ]:
# def pieplot(df):
#     data = df.value_counts().values
#     labels = df.value_counts().index

#     colors = sns.color_palette('pastel')[0:5]

#     plt.pie(data, labels = labels, autopct="%.2f%%", startangle=90, colors = colors)
#     plt.show()

In [ ]:
df['school_year'].value_counts().sort_index()

In [ ]:
pieplot(df['school_year'],0)

In [ ]:
df['semester'].value_counts().sort_index()

In [ ]:
pieplot(df['semester'],0)

In [ ]:
df['age'].value_counts().sort_index()

In [ ]:
df['gender'].value_counts().sort_index()

In [ ]:
pieplot(df['gender'],['female','male'])

In [ ]:
df['thoughts_of_dropping_out'].value_counts()

In [ ]:
pieplot(df['thoughts_of_dropping_out'],['No','Yes'])

In [ ]:
df['previous_depression_diagnosis'].value_counts()

In [ ]:
pieplot(df['previous_depression_diagnosis'],['No','Yes'])

In [ ]:
df['previous_depression_treatment'].value_counts()

In [ ]:
pieplot(df['previous_depression_treatment'],['No','Yes'])

In [ ]:
df['previous_anxiety_diagnosis'].value_counts()

In [ ]:
pieplot(df['previous_anxiety_diagnosis'],['No','Yes'])

In [ ]:
df['previous_anxiety_treatment'].value_counts()

In [ ]:
pieplot(df['previous_anxiety_treatment'],['No','Yes'])

In [ ]:
df['phq_score'].value_counts()

In [ ]:
pieplot(df['phq_score'],['No','Yes'])

In [ ]:
df['gad_score'].value_counts().sort_index(ascending=False)

In [ ]:
bins = [0, 5, 10, 15,100]
bins_label = ['no notable anxiety symptoms','mild','moderate','severe','outlier']

gad_score_cut = pd.cut(df['gad_score'], bins, right=False, labels=bins_label[:-1])
gad_score_cut.value_counts()

In [ ]:
pieplot(gad_score_cut,0)

In [ ]:
df['epw_score'].value_counts().sort_index(ascending=False)

In [ ]:
bins = [0, 8, 10, 16,100]
bins_label = ['no notable symptoms','mild','moderately','severe','outlier']

epw_score_cut = pd.cut(df['epw_score'], bins, right=False, labels=bins_label[:-1])
epw_score_cut.value_counts()

In [ ]:
pieplot(epw_score_cut,0)

In [ ]:
bins = [0, 0.3, 0.5, 0.7, 10]
bins_label = ['< 0.3','< 0.5','< 0.7',' 0.7 <','outlier']

sleep_ratio_cut = pd.cut(df['sleep_ratio'], bins, right=False, labels=bins_label[:-1])
sleep_ratio_cut.value_counts()

In [ ]:
pieplot(sleep_ratio_cut,0)

In [ ]:
df['sleep_ratio'].value_counts().sort_index()

In [ ]:
bins = [0, 18.5, 25, 30, 60]
bins_label = ['underweight','normal weight','overweight','obersity','outlier']

bmi_cut = pd.cut(df['bmi'], bins, right=False, labels=bins_label[:-1])
bmi_cut.value_counts()

In [ ]:
pieplot(bmi_cut,0)

# 3.baseline 확인

In [ ]:
def calculate_baseline_score(df):
    accuracy_score = 0
    
    from sklearn.metrics import accuracy_score
    target = 'phq_score'
    predict = df[target].mode()[0]
    y_pred = [predict] * len(df[target])
    accuracy_score = accuracy_score(df[target], y_pred)

    return accuracy_score

In [ ]:
baseline_accuracy_score = calculate_baseline_score(df)
print('baseline accuracy score:', baseline_accuracy_score)

# Q&A
Q1.해당 베이스라인 모델과 평가지표를 선택한 이유를 설명하세요.  
A1.이진클래스 분류모델에 적합한 평가지표인 최빈값인 predict를 사용하여 accuracy_score로 선택했다.

# 4.train_test_split

In [ ]:
df['phq_score'].value_counts() #샘플 불균형 확인

In [ ]:
target = df['phq_score']
features = df[df.columns.drop('phq_score')]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.25,
                                                    stratify=target, #target의 class비율에 맞춰서 분리
                                                    random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.35,
                                                    stratify=y_train, #target의 class비율에 맞춰서 분리
                                                    random_state=2)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:
print(f'{y_train.value_counts()}\n{y_val.value_counts()}\n{y_test.value_counts()}')

# 5.모델 학습

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score  

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter



In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from category_encoders import OrdinalEncoder, TargetEncoder,CatBoostEncoder
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, average_precision_score, classification_report
import warnings
warnings.filterwarnings("ignore")

## 5-1.오버샘플링
SMOTENC

In [ ]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(X_train)
X_train_preprocessed = imputer.transform(X_train)

In [ ]:
X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=X_train.columns)
X_train_preprocessed

In [ ]:
from imblearn.over_sampling import SMOTENC

categorical_features = np.delete([x for x in range(16)],[10,11,14,15]).tolist() #리스트에서 삭제['nap_duration','grades','sleep_ratio', 'bmi']

smote_nc = SMOTENC(categorical_features=categorical_features, random_state=2)
X_train_over, y_train_over = smote_nc.fit_resample(X_train_preprocessed, y_train)

In [ ]:
y_train_over.value_counts()

In [ ]:
X_train_over

## 5-2.XGBClassifier

XGBoost imbalanced data hyperparameters  
https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook  

불균형 데이터에 좋은 파라미터  
scale_pos_weight  
max_delta_step

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def xgb_origin_fit(X_train, y_train):

    pipe = Pipeline([
    ('preprocessing', make_pipeline(SimpleImputer(strategy='median'))),
    ('xgb', XGBClassifier(random_state=2,learning_rate=0.2))
    ])

    dists = {
        'xgb__n_estimators': randint(50, 300), 
        'xgb__max_depth': randint(5, 15),
        'xgb__scale_pos_weight': randint(1,10),
        'xgb__max_delta_step' : randint(1,10)
    }

    clf = RandomizedSearchCV(
        pipe, 
        param_distributions=dists, 
        n_iter=50, 
        cv=10,
        # scoring='f1',
        # scoring='balanced_accuracy',
        scoring = 'roc_auc_ovr_weighted',
        # scoring = 'f1_weighted',
        verbose=1,
        n_jobs=-1
    )

    clf.fit(X_train, y_train)
    print("Optimal Hyperparameter:", clf.best_params_)
    print("AUC:", clf.best_score_)

    return clf

In [ ]:
def predict(X, y, best_estimator):
    y_pred_proba = None

    pipe = best_estimator
    y_pred_proba = pipe.predict_proba(X)[:,1]
    
    y_pred = pipe.predict(X)

    fpr, tpr, thresholds = roc_curve(y, y_pred_proba)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    report = classification_report(y, y_pred_proba > optimal_threshold)
    return y_pred_proba, y_pred, report

In [ ]:
def plot_roc(y, y_pred_proba):
    auc_score = roc_auc_score(y, y_pred_proba)
    fpr, tpr, _ = roc_curve(y, y_pred_proba)
    baseline_fpr, baseline_tpr, _ = roc_curve(y, [0] * len(y))

    plt.style.use("ggplot")
    plt.plot(fpr, tpr, label="Model")
    plt.plot(baseline_fpr, baseline_tpr, linestyle="--", label="Baseline")
    plt.xlabel("False Positive Rate", size=12)
    plt.ylabel("True Positive Rate", size=12)
    plt.title("Receiver Operating Characteristic")
    plt.legend(prop={"size": 12})
    plt.show()
    return auc_score

In [ ]:
clf_xgb_origin = xgb_origin_fit(X_train, y_train)

In [ ]:
clf_xgb_origin.best_estimator_

In [ ]:
y_val_pred_proba, y_val_pred,report = predict(X_val, y_val, clf_xgb_origin.best_estimator_)
print(report)
auc_score = plot_roc(y_val, y_val_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_val, y_val_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_val, y_val_pred)
print('accuracy_score :', acc_score)

In [ ]:
y_test_pred_proba, y_test_pred, report = predict(X_test, y_test, clf_xgb_origin.best_estimator_)
print(report)
auc_score = plot_roc(y_test, y_test_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_test, y_test_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_test, y_test_pred)
print('accuracy_score :', acc_score)

## 5-3.XGBClassifier with SMOTENC

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def xgb_over_fit(X_train, y_train):

    pipe = Pipeline([
    ('preprocessing', make_pipeline(SimpleImputer(strategy='median'))),
    ('xgb', XGBClassifier(random_state=2,learning_rate=0.2))
    ])

    dists = {
        'xgb__n_estimators': randint(50, 300), 
        'xgb__max_depth': randint(5, 15)
    }

    clf = RandomizedSearchCV(
        pipe, 
        param_distributions=dists, 
        n_iter=50, 
        cv=10,
        # scoring='f1',
        # scoring='balanced_accuracy',
        scoring = 'roc_auc_ovr_weighted',
        # scoring = 'f1_weighted',
        verbose=1,
        n_jobs=-1
    )

    clf.fit(X_train, y_train)
    print("Optimal Hyperparameter:", clf.best_params_)
    print("AUC:", clf.best_score_)

    return clf

In [ ]:
clf_xgb_over = xgb_over_fit(X_train_over, y_train_over)

In [ ]:
clf_xgb_over

In [ ]:
y_val_pred_proba, y_val_pred, report = predict(X_val, y_val, clf_xgb_over.best_estimator_)
print(report)
auc_score = plot_roc(y_val, y_val_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_val, y_val_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_val, y_val_pred)
print('accuracy_score :', acc_score)

In [ ]:
y_test_pred_proba, y_test_pred,report = predict(X_test, y_test, clf_xgb_over.best_estimator_)
print(report)
auc_score = plot_roc(y_test, y_test_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_test, y_test_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_test, y_test_pred)
print('accuracy_score :', acc_score)

## 5-4.RandomForestClassifier

Important three techniques to improve machine learning model performance with imbalanced datasets  
https://towardsdatascience.com/working-with-highly-imbalanced-datasets-in-machine-learning-projects-c70c5f2a7b16

불균형 데이터에 좋은 파라미터  
class_weight  
min_samples_leaf

In [ ]:
def rf_origin_fit(X_train, y_train):

    pipe = Pipeline([
    ('preprocessing', make_pipeline(SimpleImputer(strategy='median'))),
    ('rf', RandomForestClassifier(random_state=2, class_weight = 'balanced'))
    ])

    dists = {
        'rf__n_estimators': randint(50, 300), 
        'rf__max_depth': randint(5, 15),
        'rf__min_samples_leaf': randint(5, 15),
        'rf__max_features': uniform(0, 1)
    }

    clf = RandomizedSearchCV(
        pipe, 
        param_distributions=dists, 
        n_iter=300, 
        cv=10,
        # scoring='f1',
        # scoring = 'balanced_accuracy',
        scoring = 'roc_auc_ovr_weighted',
        # scoring = 'f1_weighted',
        verbose=1,
        n_jobs=-1
    )

    clf.fit(X_train, y_train)
    print("Optimal Hyperparameter:", clf.best_params_)
    print("AUC:", clf.best_score_)

    return clf

In [ ]:
clf_rf_origin = rf_origin_fit(X_train, y_train)

In [ ]:
clf_rf_origin.best_estimator_

In [ ]:
y_val_pred_proba, y_val_pred,report = predict(X_val, y_val, clf_rf_origin.best_estimator_)
print(report)
auc_score = plot_roc(y_val, y_val_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_val, y_val_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_val, y_val_pred)
print('accuracy_score :', acc_score)

In [ ]:
y_test_pred_proba, y_test_pred,report = predict(X_test, y_test, clf_rf_origin.best_estimator_)
print(report)
auc_score = plot_roc(y_test, y_test_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_test, y_test_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_test, y_test_pred)
print('accuracy_score :', acc_score)

### 5-4-1.best score

In [ ]:
def rf_origin_fit(X_train, y_train):

    clf = Pipeline([
    ('preprocessing', make_pipeline(SimpleImputer(strategy='median'))),
    ('rf', RandomForestClassifier(
                                random_state=2, 
                                class_weight = 'balanced',
                                max_features=0.43,
                                min_samples_leaf=8,
                                max_depth=5,
                                n_estimators = 22))
    ])

    clf.fit(X_train, y_train)

    return clf

In [ ]:
clf_rf_origin = rf_origin_fit(X_train, y_train)

In [ ]:
y_val_pred_proba, y_val_pred,report = predict(X_val, y_val, clf_rf_origin)
print(report)
auc_score = plot_roc(y_val, y_val_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_val, y_val_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_val, y_val_pred)
print('accuracy_score :', acc_score)

In [ ]:
print('baseline accuracy score:', baseline_accuracy_score)

In [ ]:
print(acc_score > baseline_accuracy_score)

In [ ]:
y_test_pred_proba, y_test_pred,report = predict(X_test, y_test, clf_rf_origin)
print(report)
auc_score = plot_roc(y_test, y_test_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_test, y_test_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_test, y_test_pred)
print('accuracy_score :', acc_score)

In [ ]:
print(acc_score > baseline_accuracy_score)

# Q&A
Q1.모델을 학습한 후에 베이스라인보다 잘 나왔나요? 그렇지 않다면 그 이유는 무엇일까요?  
A1.베이스라인보다 높게 나왔다. 베이스라인을 넘기기위해 다양한 모델과 하이퍼 파라미터 튜닝을 시도했다.

Q2.모델 성능 개선을 위해 어떤 방법을 적용했나요? 그 방법을 선택한 이유는 무엇인가요?  
A2.모델 성능 개선을 위해 불균형 데이터의 성능향상에 좋다고 평가되는 파라미터 (class_weight, min_samples_leaf)을 적용했다.




## 5-5.RandomForestClassifier with SMOTENC

In [ ]:
def rf_over_fit(X_train, y_train):

    pipe = Pipeline([
    ('preprocessing', make_pipeline(SimpleImputer(strategy='median'))),
    ('rf', RandomForestClassifier(random_state=2))
    ])

    dists = {
        'rf__n_estimators': randint(50, 300), 
        'rf__max_depth': randint(5, 15),
        'rf__max_features': uniform(0, 1)
    }

    clf = RandomizedSearchCV(
        pipe, 
        param_distributions=dists, 
        n_iter=50, 
        cv=10,
        # scoring='f1',
        # scoring = 'balanced_accuracy',
        scoring = 'roc_auc_ovr_weighted',
        # scoring = 'f1_weighted',
        verbose=1,
        n_jobs=-1
    )

    clf.fit(X_train, y_train)
    print("Optimal Hyperparameter:", clf.best_params_)
    print("AUC:", clf.best_score_)

    return clf

In [ ]:
clf_rf_over = rf_over_fit(X_train_over, y_train_over)

In [ ]:
clf_rf_over.best_estimator_

In [ ]:
y_val_pred_proba, y_val_pred,report = predict(X_val, y_val, clf_rf_over.best_estimator_)
print(report)
auc_score = plot_roc(y_val, y_val_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_val, y_val_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_val, y_val_pred)
print('accuracy_score :', acc_score)

In [ ]:
y_test_pred_proba, y_test_pred,report = predict(X_test, y_test, clf_rf_over.best_estimator_)
print(report)
auc_score = plot_roc(y_test, y_test_pred_proba)
print("AUC:", auc_score)
f1 = f1_score(y_test, y_test_pred)
print('f1 score :', f1)
acc_score = accuracy_score(y_test, y_test_pred)
print('accuracy_score :', acc_score)

# 6.feature importance

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(clf_rf_origin, X_test, y_test,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Confusion matrix, n = {len(y_test)}', fontsize=15)
plt.show()

In [ ]:
def get_feature_importances(pipe, feature_names):
    feature_importances = np.zeros(len(feature_names))

    feature_importances = pipe._final_estimator.feature_importances_
    
    return sorted(list(zip(feature_names, feature_importances)), key=lambda x: x[1], reverse=True)

In [ ]:
feature_importances = get_feature_importances(clf_rf_origin, list(X_train.columns))
for name, importance in feature_importances:
    print(f"{name}: {importance:.2f}")

In [ ]:
pipeline = clf_rf_origin

preprocess_pipeline = pipeline.named_steps['preprocessing']
model = pipeline.named_steps['rf']

X_test_preprocessed = preprocess_pipeline.transform(X_test)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

def get_permutation_importance(X, y, model):

    permuter = PermutationImportance(
    model,
    scoring='roc_auc',
    n_iter=5,
    random_state=42
    )
    
    permuter.fit(X, y)
    return permuter

전처리 전과 후를 비교했을때 feature importance

In [ ]:
permuter = get_permutation_importance(X_test_preprocessed, y_test, model)
eli5.show_weights(permuter, top=None, feature_names=X_test.columns.tolist())
print("Top3 Important Features:")
p_i = sorted(
    list(zip(X_test.columns.tolist(), abs(permuter.feature_importances_))),
    key=lambda x: x[1],
    reverse=True,
)
print(p_i[:3])

In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_plot

def get_pdp_isolated(dataset, model, feature):
    pdp_isolated = None

    pdp_isolated = pdp_isolate(
        model=model, 
        dataset=dataset,
        model_features=dataset.columns,
        feature=feature,
        grid_type='percentile', # default='percentile', or 'equal'
        num_grid_points=10 # default=10
    )

    return pdp_isolated

In [ ]:
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=X_test.columns)
feature = sorted(p_i, key=lambda x: x[1], reverse=True)[0][0]
pdp_isolated = get_pdp_isolated(X_test_preprocessed_df, model, feature)
pdp_plot(pdp_isolated, feature_name=feature)

gad_score와 target간의 정적상관

In [ ]:
feature = sorted(p_i, key=lambda x: x[1], reverse=True)[2][0]
pdp_isolated = get_pdp_isolated(X_test_preprocessed_df, model, feature)
pdp_plot(pdp_isolated, feature_name=feature)

bmi 정상 부분인 18~25까지는 target(우울감)과 부적상관이었지만 과체중부터는 정적상관을 보인다.

In [ ]:
feature = 'grades'
pdp_isolated = get_pdp_isolated(X_test_preprocessed_df, model, feature)
pdp_plot(pdp_isolated, feature_name=feature)

grades(성적) target은 부적상관

In [ ]:
feature = 'epw_score'
pdp_isolated = get_pdp_isolated(X_test_preprocessed_df, model, feature)
pdp_plot(pdp_isolated, feature_name=feature)

epw_score 중증 기준인 15점부터 target과 정적상관

In [ ]:
feature = 'age'
pdp_isolated = get_pdp_isolated(X_test_preprocessed_df, model, feature)
pdp_plot(pdp_isolated, feature_name=feature)

In [ ]:
feature = 'bmi'
pdp_isolated = get_pdp_isolated(X_test_preprocessed_df, model, feature)
pdp_plot(pdp_isolated, feature_name=feature)

나이는 target과 상관이없다

In [ ]:
from pdpbox.pdp import pdp_interact, pdp_interact_plot

def get_pdp_interaction(dataset, model, features):

    pdp_interaction = pdp_interact(
    model=model, 
    dataset=dataset,
    model_features=dataset.columns, 
    features=features
    )
    return pdp_interaction

In [ ]:
features = ['epw_score','nap_duration']
pdp_interaction = get_pdp_interaction(
    X_test_preprocessed_df, model, features
)
pdp_interact_plot(pdp_interaction, feature_names=features, plot_type="grid")

In [ ]:
features = ['epw_score','gad_score']
pdp_interaction = get_pdp_interaction(
    X_test_preprocessed_df, model, features
)
pdp_interact_plot(pdp_interaction, feature_names=features, plot_type="grid")

In [ ]:
features = ['bmi','gad_score']
pdp_interaction = get_pdp_interaction(
    X_test_preprocessed_df, model, features
)
pdp_interact_plot(pdp_interaction, feature_names=features, plot_type="grid")

In [ ]:
features = ['bmi','epw_score']
pdp_interaction = get_pdp_interaction(
    X_test_preprocessed_df, model, features
)
pdp_interact_plot(pdp_interaction, feature_names=features, plot_type="grid")

# 7.shap

In [ ]:
import shap

In [ ]:
pipeline = clf_rf_origin
model = pipeline.named_steps['rf']

X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=X_test.columns)
explainer = shap.TreeExplainer(model)
shap_values = np.array(explainer.shap_values(X_test_preprocessed))

In [ ]:
shap.summary_plot(shap_values[1], X_test_preprocessed, plot_type= 'bar' )

In [ ]:
shap.summary_plot(shap_values[1],X_test_preprocessed,plot_size=(15,10))

In [ ]:
# shap.summary_plot(shap_values[0],X_test_preprocessed)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0],feature_names=X_test_preprocessed.columns) 